In [16]:
pip install datasets
pip install keybert

     -------------------------------------- 468.7/468.7 kB 2.7 MB/s eta 0:00:00
     -------------------------------------- 132.9/132.9 kB 4.0 MB/s eta 0:00:00
     -------------------------------------- 323.6/323.6 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 20.6/20.6 MB 6.9 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB 3.1 MB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4
Note: you may need to restart the kernel to use updated packages.


In [40]:
import pandas as pd

In [34]:
# get the dataset only for keyphrase generation
dataset = load_dataset("midas/duc2001", "generation")

print("Samples for Keyphrase Generation")

# sample from the test split
print("Sample from test data split")
test_sample = dataset["test"][0]
print("Fields in the sample: ", [key for key in test_sample.keys()])
print("Tokenized Document: ", test_sample["document"])
print("Extractive/present Keyphrases: ", test_sample["extractive_keyphrases"])
print("Abstractive/absent Keyphrases: ", test_sample["abstractive_keyphrases"])
print("\n-----------\n")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset duc2001 downloaded and prepared to C:/Users/Alicia/.cache/huggingface/datasets/midas___duc2001/generation/0.0.1/7888b46165d8a58f49f00e28410b46b1f22fabfd72a9e89f3e80a4e2d27e4a9b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Samples for Keyphrase Generation
Sample from test data split
Fields in the sample:  ['id', 'document', 'extractive_keyphrases', 'abstractive_keyphrases']
Tokenized Document:  ['Here', ',', 'at', 'a', 'glance', ',', 'are', 'developments', 'today', 'involving', 'the', 'crash', 'of', 'Pan', 'American', 'World', 'Airways', 'Flight', '103', 'Wednesday', 'night', 'in', 'Lockerbie', ',', 'Scotland', ',', 'that', 'killed', 'all', '259', 'people', 'aboard', 'and', 'more', 'than', '20', 'people', 'on', 'the', 'ground', ':']
Extractive/present Keyphrases:  ['pan american world airways flight 103', 'crash', 'lockerbie']
Abstractive/absent Keyphrases:  ['terrorist threats', 'widespread wreckage', 'radical palestinian faction', 'terrorist bombing', 'bomb threat', 'sabotage']

-----------



In [42]:
def drop_column(dataset):
    del dataset['abstractive_keyphrases']
    return dataset

# Para eliminar la columna que no necesitamos
dataset = dataset.map(drop_column)

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

Hasta aquí tenemos un dataset con las 3 columnas de interés:

In [113]:
dataset

DatasetDict({
    test: Dataset({
        features: ['id', 'document', 'extractive_keyphrases'],
        num_rows: 308
    })
})

A continuación juntamos las palabras de 'document' porque estan separadas por comas, y para cada texto resultante implementamos keybert. Guardamos las keywords y el score de cada texto

In [102]:
from keybert import KeyBERT
kw_model = KeyBERT()

In [105]:
keywords = []
for i in range(len(dataset['test'])):
    joined_text = " ".join(dataset['test'][i]['document'])
    keywords_row = []
    for keyword_info in kw_model.extract_keywords(joined_text, keyphrase_ngram_range=(1, 1), stop_words=None):
        keyword, score = keyword_info  # extraer palabra clave y puntuación
        keywords_row.append({'keyword': keyword, 'score': score}) # guardar información en un diccionario
    keywords.append(keywords_row) # guardar información de palabras clave para cada fila


In [112]:
keywords[0] #prueba

[{'keyword': 'scott', 'score': 0.3601},
 {'keyword': 'justice', 'score': 0.3453},
 {'keyword': 'missouri', 'score': 0.3439},
 {'keyword': 'unconstitutional', 'score': 0.3265},
 {'keyword': 'constitutional', 'score': 0.3133}]

In [110]:
len(keywords)==len(dataset['test']) #comprovación para ver que se ha hecho para todos los textos

True

Si miramos las extractive keyphrases, vemos que la mayoría de ellas tienen 1 o 2 N-gramas. Si ahora en vez de keywords queremos extraer kephrases de 2 palabras tenemos que cambiar el parámetro keyphrase_ngram_range:

In [114]:
keyphrases = []
for i in range(len(dataset['test'])):
    joined_text = " ".join(dataset['test'][i]['document'])
    keywords_row = []
    for keyword_info in kw_model.extract_keywords(joined_text, keyphrase_ngram_range=(1, 2), stop_words=None):
        keyword, score = keyword_info  # extraer palabra clave y puntuación
        keywords_row.append({'keyword': keyword, 'score': score}) # guardar información en un diccionario
    keyphrases.append(keywords_row) # guardar información de palabras clave para cada fila

In [115]:
keyphrases[0] #prueba

[{'keyword': 'flight 103', 'score': 0.4514},
 {'keyword': 'the crash', 'score': 0.4259},
 {'keyword': 'world airways', 'score': 0.4206},
 {'keyword': '259 people', 'score': 0.4203},
 {'keyword': 'airways flight', 'score': 0.4154}]

Después de observar que se puede modificar el tamaño de N-gramas, keybert nos permite modificar otras opciones. Una de  ellas es la Max Sum Distance, la cual coge todas las top_n combinaciones de las 2x top_n palabras que hay, y extrae las combinaciones que son menos similares entre ellas a partir de la similitud de cosinus. Ej:

In [116]:
keyphrases2 = []
for i in range(len(dataset['test'])):
    joined_text = " ".join(dataset['test'][i]['document'])
    keywords_row = []
    for keyword_info in kw_model.extract_keywords(joined_text, keyphrase_ngram_range=(1, 2), stop_words=None, use_maxsum=True, 
                                                  nr_candidates=20, top_n=5):
        keyword, score = keyword_info  # extraer palabra clave y puntuación
        keywords_row.append({'keyword': keyword, 'score': score}) # guardar información en un diccionario
    keyphrases2.append(keywords_row) # guardar información de palabras clave para cada fila

In [117]:
keyphrases2[0] #prueba

[{'keyword': 'lockerbie scotland', 'score': 0.2471},
 {'keyword': 'pan american', 'score': 0.2887},
 {'keyword': 'killed all', 'score': 0.3371},
 {'keyword': 'airways flight', 'score': 0.4154},
 {'keyword': '259 people', 'score': 0.4203}]

Una vez añadidos estos parámetros, también podemos incorporar el uso de Maximal Marginal Relevance, que sirve para crear keywords/keyphrases y también se basa en la similitud de cosinus. Sirve para diversificar los resultados. Podemos probar con alta diversidad (0.7) o con baja diversidad (0.2). Ej:

In [118]:
keyphrases3 = []
for i in range(len(dataset['test'])):
    joined_text = " ".join(dataset['test'][i]['document'])
    keywords_row = []
    for keyword_info in kw_model.extract_keywords(joined_text, keyphrase_ngram_range=(1, 2), stop_words=None, use_maxsum=True, 
                                                  nr_candidates=20, top_n=5, diversity=0.7):
        keyword, score = keyword_info  # extraer palabra clave y puntuación
        keywords_row.append({'keyword': keyword, 'score': score}) # guardar información en un diccionario
    keyphrases3.append(keywords_row) # guardar información de palabras clave para cada fila

In [119]:
keyphrases3[0] #prueba, vemos que salen las mismas q antes (almenos en este caso)

[{'keyword': 'lockerbie scotland', 'score': 0.2471},
 {'keyword': 'pan american', 'score': 0.2887},
 {'keyword': 'killed all', 'score': 0.3371},
 {'keyword': 'airways flight', 'score': 0.4154},
 {'keyword': '259 people', 'score': 0.4203}]

In [120]:
keyphrases2[10]

[{'keyword': 'officials about', 'score': 0.3666},
 {'keyword': 'boeing 737s', 'score': 0.3767},
 {'keyword': 'crashed sunday', 'score': 0.3788},
 {'keyword': 'ordered inspections', 'score': 0.3837},
 {'keyword': 'engine warning', 'score': 0.3888}]

In [121]:
keyphrases3[10]

[{'keyword': 'officials about', 'score': 0.3666},
 {'keyword': 'boeing 737s', 'score': 0.3767},
 {'keyword': 'crashed sunday', 'score': 0.3788},
 {'keyword': 'ordered inspections', 'score': 0.3837},
 {'keyword': 'engine warning', 'score': 0.3888}]